In [2]:
def Preprocess(file):
    array = []
    for line in f:
        array.append(line)
    xc = [0]*300
    yc = [0]*300
    maxrows = 300
    maxcols = 300
    Cost = [([0] * maxcols) for row in range(maxrows) ]

    dim = int(array[0])
    for it in range(1, dim+1):
        p = array[it]
        tokens = p.split()
        xc[it-1] = float(tokens[0])
        yc[it-1] = float(tokens[1])
    
    for i in range(0, dim):
        for j in range(0, dim):
            Cost[i][j] = pow((pow(xc[i]-xc[j], 2) + pow(yc[i] - yc[j], 2)), 0.5)
        
    for i in range(0, dim):
        for j in range(0, dim):
            if(i==j):
                Cost[i][j] = 99999
    return Cost, xc, yc, dim


In [4]:
from gurobipy import *
import string
import time

f = open("C:\\Users\\Prasangsha\\Downloads\\TSP_instance_n_15_s_1.dat")
Cost, xc, yc, dim = Preprocess(f)

startTime = time.time()
TSP_Q = Model('TSP_Quadratic')


w = [[[0 for i in range(dim)] for j in range(dim)] for k in range(dim)] 
x = [[0 for i in range(dim)] for j in range(dim)]

for i in range(0, dim):
    for j in range(0, dim):
        for t in range(0, dim):
            w[i][j][t] = TSP_Q.addVar(vtype=GRB.BINARY, name = 'w[][][]'.format(i,j,t))
            
for i in range(0, dim):
    for j in range(0, dim):
        x[i][j] = TSP_Q.addVar(vtype=GRB.BINARY, name = 'x[][]'.format(i,j))
    
TSP_Q.update()

TSP_Q.setObjective(quicksum(quicksum(Cost[i][j]*(quicksum (w[i][j][k] for k in range(0, dim-1))) for i in range(dim)) for j in range(dim)) + quicksum(quicksum (Cost[i][j]*w[i][j][dim-1] for j in range(0, dim)) for i in range(0, dim)), GRB.MINIMIZE)



for i in range(0, dim):
    TSP_Q.addConstr(quicksum(x[i][j] for j in range(0, dim) ) == 1)
    
for j in range(0, dim):
    TSP_Q.addConstr(quicksum(x[i][j] for i in range(0, dim) ) == 1)
    
    
for i in range(0, dim):
    for j in range(0, dim):
        for k in range(0, dim-1):
            TSP_Q.addConstr(w[i][j][k] >= x[i][k] + x[j][k+1] -1)
            
for i in range(0, dim):
    for j in range(0, dim):
        TSP_Q.addConstr(w[i][j][dim-1] >= x[i][dim-1] + x[j][0] -1)
        
TSP_Q.optimize()

executionTime = (time.time() - startTime)
print("The execution time is")
print(executionTime)


Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 3405 rows, 3600 columns and 10575 nonzeros
Model fingerprint: 0x1dea6c12
Variable types: 0 continuous, 3600 integer (3600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 111.0339672
Presolve removed 225 rows and 225 columns
Presolve time: 0.02s
Presolved: 3180 rows, 3375 columns, 9900 nonzeros
Variable types: 0 continuous, 3375 integer (3375 binary)

Root relaxation: objective 0.000000e+00, 121 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   41  111.03397    0.00000   100%     -    0s
H    0     0                     109.1179711    0.00000   100%     -    0s
     0     0    0.00000    0   40  109.117